In [7]:
import xarray as xr
import rioxarray as rioxr
import numpy as np
import os

def mask_and_fill(tiff_files, mask_file, output_files, chunk_size=500):
    """
    Masks and fills gaps in a list of TIFF files based on a specified mask TIFF file.

    Args:
        tiff_files (list): List of input TIFF file paths.
        mask_file (str): Path to the mask TIFF file.
        output_files (list): List of output TIFF file paths.
        chunk_size (int): Size of chunks for processing large data.
    """
    # Validate file paths
    if not os.path.exists(mask_file):
        raise FileNotFoundError(f"Mask file not found: {mask_file}")
    for tiff_file in tiff_files:
        if not os.path.exists(tiff_file):
            raise FileNotFoundError(f"Input file not found: {tiff_file}")

    # Load the mask file
    print(f"Loading mask file: {mask_file}")
    mask = rioxr.open_rasterio(mask_file).squeeze()  # Remove extra dimensions if present
    print("Mask dimensions:", mask.shape)

    for i, tiff_file in enumerate(tiff_files):
        print(f"Processing file: {tiff_file}")
        try:
            # Load the current TIFF file
            ds = rioxr.open_rasterio(tiff_file).squeeze()
            print(f"File dimensions: {ds.shape}")

            # Process in chunks if the data is large
            for start in range(0, ds.shape[0], chunk_size):
                chunk = ds[start:start + chunk_size]
                print(f"Processing chunk {start} to {start + chunk_size}")

                # Apply the mask
                masked_data = np.where(mask.data == 1, chunk.data, np.nan)

                # Fill gaps in the data (fill with mean value)
                filled_data = xr.DataArray(masked_data, coords=chunk.coords, dims=chunk.dims).fillna(masked_data.mean())

                # Convert back to xarray dataset
                chunk_masked_filled = xr.Dataset({f"band_{i+1}": filled_data}, coords=chunk.coords)

                # Save the chunk
                chunk_masked_filled.rio.to_raster(output_files[i])
                print(f"Saved processed chunk {start} to {start + chunk_size} for file {tiff_file}.")

        except Exception as e:
            print(f"Error processing file {tiff_file}: {e}")

# Paths to the input TIFF files
tiff_files = [
    r"D:\merged_transitionmap\merge_transitionmap.tif",
    r"D:\merge_wetland\merge_wetland.tif",
    r"C:\Users\rubab\Desktop\GWE_codes\WaterTable_dataUSGS\masked_binary_image_30m.tif"
]

# Path to the mask TIFF file
mask_file = r"C:\Users\rubab\Desktop\GWE_codes\Mask_CONUS_SentbyLaura\CONUS2_0_Final250m_Mask.tif"

# Paths to the output TIFF files
output_files = [
    r"D:\merged_transitionmap\merge_transitionmap_masked_filled.tif",
    r"D:\merge_wetland\merge_wetland_masked_filled.tif",
    r"C:\Users\rubab\Desktop\GWE_codes\WaterTable_dataUSGS\masked_binary_image_30m_masked_filled.tif"
]

# Process the files
mask_and_fill(tiff_files, mask_file, output_files)


Loading mask file: C:\Users\rubab\Desktop\GWE_codes\Mask_CONUS_SentbyLaura\CONUS2_0_Final250m_Mask.tif
Mask dimensions: (13024, 17768)
Processing file: D:\merged_transitionmap\merge_transitionmap.tif
File dimensions: (120522, 247128)
Processing chunk 0 to 500
Error processing file D:\merged_transitionmap\merge_transitionmap.tif: operands could not be broadcast together with shapes (13024,17768) (500,247128) () 
Processing file: D:\merge_wetland\merge_wetland.tif
File dimensions: (4, 97277, 154258)
Processing chunk 0 to 500
Error processing file D:\merge_wetland\merge_wetland.tif: operands could not be broadcast together with shapes (13024,17768) (4,97277,154258) () 
Processing file: C:\Users\rubab\Desktop\GWE_codes\WaterTable_dataUSGS\masked_binary_image_30m.tif
File dimensions: (833, 1966)
Processing chunk 0 to 500
Error processing file C:\Users\rubab\Desktop\GWE_codes\WaterTable_dataUSGS\masked_binary_image_30m.tif: operands could not be broadcast together with shapes (13024,17768) (

In [24]:
import rioxarray as rioxr
import numpy as np
import os

def process_in_chunks(ds, mask, output_file, chunk_size):
    """
    Process the dataset in chunks to apply mask and fill gaps.

    Args:
        ds (xarray.DataArray): Dataset to process.
        mask (xarray.DataArray): Mask to apply.
        output_file (str): Path to save the processed file.
        chunk_size (int): Chunk size to process the data.
    """
    print(f"Processing in chunks for: {output_file}")
    with rioxr.open_rasterio(output_file, mode="w", dtype=ds.dtype) as dst:
        for y_start in range(0, ds.shape[1], chunk_size):
            for x_start in range(0, ds.shape[2], chunk_size):
                y_end = min(y_start + chunk_size, ds.shape[1])
                x_end = min(x_start + chunk_size, ds.shape[2])

                # Extract chunk
                chunk = ds[:, y_start:y_end, x_start:x_end]
                mask_chunk = mask[y_start:y_end, x_start:x_end]

                # Apply mask and fill gaps
                processed_chunk = np.where(mask_chunk == 1, chunk, 0)
                dst.write(processed_chunk, window=((y_start, y_end), (x_start, x_end)))

    print(f"Processed and saved: {output_file}")


def mask_and_fill(tiff_files, mask_file, output_files, chunk_size=5000):
    """
    Masks and fills gaps in a list of TIFF files based on a specified mask TIFF file.

    Args:
        tiff_files (list): List of input TIFF file paths.
        mask_file (str): Path to the mask TIFF file.
        output_files (list): List of output TIFF file paths.
        chunk_size (int): Chunk size for processing.
    """
    # Validate file paths
    if not os.path.exists(mask_file):
        raise FileNotFoundError(f"Mask file not found: {mask_file}")
    for tiff_file in tiff_files:
        if not os.path.exists(tiff_file):
            raise FileNotFoundError(f"Input file not found: {tiff_file}")

    # Load the mask file
    print(f"Loading mask file: {mask_file}")
    mask = rioxr.open_rasterio(mask_file).squeeze()  # Load mask as DataArray
    mask = mask.rio.reproject(mask.rio.crs)
    print(f"Mask dimensions: {mask.shape}")

    for i, tiff_file in enumerate(tiff_files):
        print(f"Processing file: {tiff_file}")
        try:
            # Load the current TIFF file
            ds = rioxr.open_rasterio(tiff_file, chunks=True).squeeze()  # Use chunked loading

            # Reproject to match the mask dimensions
            if ds.rio.shape != mask.shape:
                print(f"Reprojecting {tiff_file} to match mask dimensions.")
                ds = ds.rio.reproject_match(mask)

            # Process the data in chunks
            process_in_chunks(ds, mask, output_files[i], chunk_size)

        except Exception as e:
            print(f"Error processing {tiff_file}: {e}")


# Paths to the input TIFF files
tiff_files = [
    r"D:\merged_transitionmap\merge_transitionmap.tif",
    r"D:\merge_wetland\merge_wetland.tif",
    r"C:\Users\rubab\Desktop\GWE_codes\WaterTable_dataUSGS\masked_binary_image_30m.tif"
]

# Path to the mask TIFF file
mask_file = r"C:\Users\rubab\Desktop\GWE_codes\Mask_CONUS_SentbyLaura\CONUS2_0_Final250m_Mask.tif"

# Paths to the output TIFF files
output_files = [
    r"D:\merged_transitionmap\merge_transitionmap_masked_filled.tif",
    r"D:\merge_wetland\merge_wetland_masked_filled.tif",
    r"C:\Users\rubab\Desktop\GWE_codes\WaterTable_dataUSGS\masked_binary_image_30m_masked_filled.tif"
]

# Process the files
mask_and_fill(tiff_files, mask_file, output_files, chunk_size=1000)


Loading mask file: C:\Users\rubab\Desktop\GWE_codes\Mask_CONUS_SentbyLaura\CONUS2_0_Final250m_Mask.tif
Mask dimensions: (13024, 17768)
Processing file: D:\merged_transitionmap\merge_transitionmap.tif
Error processing D:\merged_transitionmap\merge_transitionmap.tif: Automatic chunking requires dask >= 0.18.0
Processing file: D:\merge_wetland\merge_wetland.tif
Error processing D:\merge_wetland\merge_wetland.tif: Automatic chunking requires dask >= 0.18.0
Processing file: C:\Users\rubab\Desktop\GWE_codes\WaterTable_dataUSGS\masked_binary_image_30m.tif
Error processing C:\Users\rubab\Desktop\GWE_codes\WaterTable_dataUSGS\masked_binary_image_30m.tif: Automatic chunking requires dask >= 0.18.0


In [35]:
import rasterio
from rasterio.windows import Window
import os

def process_chunk(data, x_offset, y_offset):
    """
    Example function to process a chunk of raster data.
    Replace this with your actual processing logic.
    """
    # Example: Normalize data (scale between 0 and 1)
    processed = (data - data.min()) / (data.max() - data.min() + 1e-5)  # Avoid division by zero
    print(f"Processed chunk at offset ({x_offset}, {y_offset})")
    return processed

def process_raster_in_chunks(input_path, output_path, chunk_size):
    """
    Process a large raster file in manageable chunks and write the output.
    Args:
        input_path (str): Path to the input raster file.
        output_path (str): Path to save the processed raster.
        chunk_size (int): Size of the chunk (in pixels).
    """
    # Ensure the input file exists
    if not os.path.exists(input_path):
        raise FileNotFoundError(f"Input raster file not found: {input_path}")

    with rasterio.open(input_path) as src:
        # Copy metadata for the output file
        profile = src.profile
        profile.update(dtype='float32', compress='lzw')

        # Create the output file
        with rasterio.open(output_path, 'w', **profile) as dst:
            # Process the raster in chunks
            for y_offset in range(0, src.height, chunk_size):
                for x_offset in range(0, src.width, chunk_size):
                    # Define a window for the chunk
                    window = Window(x_offset, y_offset,
                                    min(chunk_size, src.width - x_offset),
                                    min(chunk_size, src.height - y_offset))
                    # Read data for the current chunk
                    data = src.read(1, window=window)

                    # Process the chunk
                    processed_data = process_chunk(data, x_offset, y_offset)

                    # Write the processed data to the output raster
                    dst.write(processed_data.astype('float32'), 1, window=window)

# Specify paths to your raster files
input_raster = r"D:\merge_wetland\merge_wetland.tif"
output_raster = r"D:\processed_output\merge_processed.tif"

# Ensure the output directory exists
output_dir = os.path.dirname(output_raster)
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Process the raster in 1000x1000 pixel chunks
try:
    process_raster_in_chunks(input_raster, output_raster, chunk_size=1000)
    print(f"Processing completed successfully! Output saved to: {output_raster}")
except Exception as e:
    print(f"An error occurred: {e}")


Processed chunk at offset (0, 0)
Processed chunk at offset (1000, 0)
Processed chunk at offset (2000, 0)
Processed chunk at offset (3000, 0)
Processed chunk at offset (4000, 0)
Processed chunk at offset (5000, 0)
Processed chunk at offset (6000, 0)
Processed chunk at offset (7000, 0)
Processed chunk at offset (8000, 0)
Processed chunk at offset (9000, 0)
Processed chunk at offset (10000, 0)
Processed chunk at offset (11000, 0)
Processed chunk at offset (12000, 0)
Processed chunk at offset (13000, 0)
Processed chunk at offset (14000, 0)
Processed chunk at offset (15000, 0)
Processed chunk at offset (16000, 0)
Processed chunk at offset (17000, 0)
Processed chunk at offset (18000, 0)
Processed chunk at offset (19000, 0)
Processed chunk at offset (20000, 0)
Processed chunk at offset (21000, 0)
Processed chunk at offset (22000, 0)
Processed chunk at offset (23000, 0)
Processed chunk at offset (24000, 0)
Processed chunk at offset (25000, 0)
Processed chunk at offset (26000, 0)
Processed chun

In [36]:
import rasterio
from rasterio.windows import Window
import numpy as np
import os

def process_chunk(data, x_offset, y_offset, nodata):
    """
    Example function to process a chunk of raster data.
    Args:
        data (np.ndarray): The data for the chunk.
        x_offset (int): The x-offset of the chunk in the raster.
        y_offset (int): The y-offset of the chunk in the raster.
        nodata (float): The nodata value of the raster.

    Returns:
        np.ndarray: Processed data for the chunk.
    """
    # Handle nodata values
    valid_data = data[data != nodata]
    
    if valid_data.size > 0:
        # Normalize only valid data
        normalized = (data - valid_data.min()) / (valid_data.max() - valid_data.min() + 1e-5)
        normalized[data == nodata] = nodata  # Preserve nodata values
    else:
        # If no valid data, return the same chunk
        normalized = np.full_like(data, nodata)

    print(f"Processed chunk at offset ({x_offset}, {y_offset})")
    return normalized

def process_raster_in_chunks(input_path, output_path, chunk_size):
    """
    Process a large raster file in manageable chunks and write the output.
    Args:
        input_path (str): Path to the input raster file.
        output_path (str): Path to save the processed raster.
        chunk_size (int): Size of the chunk (in pixels).
    """
    # Ensure the input file exists
    if not os.path.exists(input_path):
        raise FileNotFoundError(f"Input raster file not found: {input_path}")

    with rasterio.open(input_path) as src:
        # Get nodata value from source
        nodata = src.nodata if src.nodata is not None else -9999

        # Copy metadata for the output file
        profile = src.profile
        profile.update(dtype='float32', nodata=nodata, compress='lzw')

        # Create the output file
        with rasterio.open(output_path, 'w', **profile) as dst:
            # Process the raster in chunks
            for y_offset in range(0, src.height, chunk_size):
                for x_offset in range(0, src.width, chunk_size):
                    # Define a window for the chunk
                    window = Window(x_offset, y_offset,
                                    min(chunk_size, src.width - x_offset),
                                    min(chunk_size, src.height - y_offset))
                    # Read data for the current chunk
                    data = src.read(1, window=window)

                    # Process the chunk
                    processed_data = process_chunk(data, x_offset, y_offset, nodata)

                    # Write the processed data to the output raster
                    dst.write(processed_data.astype('float32'), 1, window=window)

# Specify paths to your raster files
input_raster = r"D:\merge_wetland\merge_wetland.tif"
output_raster = r"D:\processed_output\merge_processed.tif"

# Ensure the output directory exists
output_dir = os.path.dirname(output_raster)
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Process the raster in 1000x1000 pixel chunks
try:
    process_raster_in_chunks(input_raster, output_raster, chunk_size=1000)
    print(f"Processing completed successfully! Output saved to: {output_raster}")
except Exception as e:
    print(f"An error occurred: {e}")


Processed chunk at offset (0, 0)
Processed chunk at offset (1000, 0)
Processed chunk at offset (2000, 0)
Processed chunk at offset (3000, 0)
Processed chunk at offset (4000, 0)
Processed chunk at offset (5000, 0)
Processed chunk at offset (6000, 0)
Processed chunk at offset (7000, 0)
Processed chunk at offset (8000, 0)
Processed chunk at offset (9000, 0)
Processed chunk at offset (10000, 0)
Processed chunk at offset (11000, 0)
Processed chunk at offset (12000, 0)
Processed chunk at offset (13000, 0)
Processed chunk at offset (14000, 0)
Processed chunk at offset (15000, 0)
Processed chunk at offset (16000, 0)
Processed chunk at offset (17000, 0)
Processed chunk at offset (18000, 0)
Processed chunk at offset (19000, 0)
Processed chunk at offset (20000, 0)
Processed chunk at offset (21000, 0)
Processed chunk at offset (22000, 0)
Processed chunk at offset (23000, 0)
Processed chunk at offset (24000, 0)
Processed chunk at offset (25000, 0)
Processed chunk at offset (26000, 0)
Processed chun